In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from glob import glob

import keras

c:\Users\huang\miniconda3\envs\gpu-ml-base\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
c:\Users\huang\miniconda3\envs\gpu-ml-base\Lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.6 when it was built against 1.14.5, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [ ]:
def load_tcr_dataset_from_dir(data_dir, batch_size=100, shuffle=True):
    # Collect Files
    df_rep = pd.DataFrame(glob(os.path.join(data_dir, '*/*.tsv')), columns=['filepath'])
    df_rep[['HLA', 'Antigen']] = df_rep['filepath'].str.extract(r'/([^/-]+)-([^/-]+)/[^/]+$')

    # Load TSV into df
    df_tcr = []
    for idx in df_rep.index:
        df = pd.read_csv(df_rep.loc[idx, 'filepath'], sep='\t')
        df['index'] = idx
        df[['HLA', 'Antigen']] = df_rep.loc[idx, ['HLA', 'Antigen']].values
        df_tcr.append(df)
    df_tcr = pd.concat(df_tcr)
    df_tcr['Antigen'] = df_tcr['Antigen'].astype('category')

    # Embedding
    idx = ~df_tcr[['aminoAcid', 'Antigen']].isna().any(axis=1)
    X = df_tcr.loc[idx, 'aminoAcid'].values
    y = df_tcr.loc[idx, 'Antigen'].cat.codes.values
    label_map = dict(enumerate(df_tcr.loc[idx, 'Antigen'].cat.categories))

    max_length = max(map(len, X))
    vocab = sorted(set(''.join(X)))
    aa_dict = {aa: i + 1 for i, aa in enumerate(vocab)}

    # Padding
    X_encoded = np.zeros((len(X), max_length), dtype=np.int32)
    for i, seq in enumerate(X):
        for j, aa in enumerate(seq[:max_length]):
            X_encoded[i, j] = aa_dict.get(aa, 0)

    # Create TensorFlow dataset
    ds = tf.data.Dataset.from_tensor_slices((X_encoded, y))
    ds = ds.map(lambda x, y: (x, tf.one_hot(y, len(label_map))))
    if shuffle:
        ds = ds.shuffle(len(X_encoded))
    ds = ds.batch(batch_size, drop_remainder = True).prefetch(tf.data.AUTOTUNE)

    return ds, aa_dict, label_map

In [3]:
data_dir = '/projects/deeptcr/DeepTCR/Data/Human_Antigens'

In [ ]:
tfds, aa_dict, label_map= load_tcr_dataset_from_dir(data_dir)

ValueError: No objects to concatenate

In [ ]:
tfds

<_PrefetchDataset element_spec=(TensorSpec(shape=(100, 23), dtype=tf.int32, name=None), TensorSpec(shape=(100, 7), dtype=tf.float32, name=None))>

In [ ]:
aa_dict

{'A': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'K': 9,
 'L': 10,
 'M': 11,
 'N': 12,
 'P': 13,
 'Q': 14,
 'R': 15,
 'S': 16,
 'T': 17,
 'V': 18,
 'W': 19,
 'Y': 20}

In [ ]:
label_map

{0: 'CTELKLSDY',
 1: 'GILGFVFTL',
 2: 'GLCTLVAML',
 3: 'LPRRSGAAGA',
 4: 'NLVPMVATV',
 5: 'TPRVTGGGAM',
 6: 'VTEHDTLLY'}

In [ ]:
# CDR3 AA encoding
tensors = []
tensors.append(keras.layers.Input(shape=(23, ),  dtype=tf.uint32))
tensors.append(keras.layers.Embedding(input_dim=len(aa_dict) + 1, output_dim=64, mask_zero=True)(tensors[-1]))
tensors.append(keras.layers.Conv1D(filters=64, kernel_size=9, strides=1, activation=keras.activations.relu)(tensors[-1]))
tensors.append(keras.layers.Conv1D(filters=128, kernel_size=7, strides=2, activation=keras.activations.relu)(tensors[-1]))
tensors.append(keras.layers.Conv1D(filters=256, kernel_size=5, strides=1, activation=None)(tensors[-1])[:, 0, :])
encoder = keras.Model(inputs=tensors[0], outputs=tensors[-1], name='encoder')

In [ ]:
# VDJ usage encoding
tensors = []
tensors.append(keras.layers.Input(shape=(23, ),  dtype=tf.uint32))
tensors.append(keras.layers.Embedding(input_dim=len(aa_dict) + 1, output_dim=64, mask_zero=True)(tensors[-1]))
tensors.append(keras.layers.Conv1D(filters=64, kernel_size=9, strides=1, activation=keras.activations.relu)(tensors[-1]))
tensors.append(keras.layers.Conv1D(filters=128, kernel_size=7, strides=2, activation=keras.activations.relu)(tensors[-1]))
tensors.append(keras.layers.Conv1D(filters=256, kernel_size=5, strides=1, activation=None)(tensors[-1])[:, 0, :])
encoder = keras.Model(inputs=tensors[0], outputs=tensors[-1], name='encoder')

In [ ]:
tensors = []
tensors.append(keras.layers.Input(shape=(encoder.output_shape[-1], ), dtype=tf.float32))
tensors.append(keras.layers.Dropout(rate=0.1)(tensors[-1]))
tensors.append(keras.layers.Dense(units=128, activation=keras.activations.relu)(tensors[-1]))
tensors.append(keras.layers.Dropout(rate=0.05)(tensors[-1]))
tensors.append(keras.layers.Dense(units=64, activation=keras.activations.relu)(tensors[-1]))
tensors.append(keras.layers.Dense(units=len(label_map), activation=keras.activations.sigmoid)(tensors[-1]))
classifier = keras.Model(inputs=tensors[0], outputs=tensors[-1], name='classifer')

In [ ]:
tensors = []
tensors.append(keras.layers.Input(shape=encoder.input_shape[1:],  dtype=tf.uint32))
tensors.append(encoder(tensors[-1]))
tensors.append(classifier(tensors[-1]))
model = keras.Model(inputs=tensors[0], outputs=tensors[-1], name='full_model')

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=keras.losses.BinaryCrossentropy(from_logits=False)
)

In [ ]:
model.fit(tfds, epochs=100)

Epoch 1/100


20/20 [==============================] - 1s 6ms/step - loss: 0.4667
Epoch 2/100
20/20 [==============================] - 0s 2ms/step - loss: 0.3591
Epoch 3/100
20/20 [==============================] - 0s 2ms/step - loss: 0.3535
Epoch 4/100
20/20 [==============================] - 0s 2ms/step - loss: 0.3516
Epoch 5/100
20/20 [==============================] - 0s 2ms/step - loss: 0.3458
Epoch 6/100
20/20 [==============================] - 0s 3ms/step - loss: 0.3376
Epoch 7/100
20/20 [==============================] - 0s 2ms/step - loss: 0.3269
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.3156
Epoch 9/100
20/20 [==============================] - 0s 2ms/step - loss: 0.3044
Epoch 10/100
20/20 [==============================] - 0s 2ms/step - loss: 0.2884
Epoch 11/100
20/20 [==============================] - 0s 3ms/step - loss: 0.2725
Epoch 12/100
20/20 [==============================] - 0s 3ms/step - loss: 0.2529
Epoch 13/100
20/20 [=============================

In [ ]:
b = next(iter(tfds))

In [ ]:
b[0].shape, b[1].shape

(TensorShape([100, 23]), TensorShape([100, 7]))